In [ ]:
from requests import get
from pandas import DataFrame, concat, offsets
from datetime import datetime
from random import randint
from json import loads
from time import sleep

In [ ]:
def map_ticker(ticker = 'TSLA'):
    url = 'https://www.sec.gov/files/company_tickers.json'

    headers = {
        'User-Agent': 'api@gmail.com',
    }

    res = get(url, headers=headers)
    res = loads(res.content.decode('utf-8'))
    ticker_list = DataFrame(res).transpose()

    cik = str(ticker_list[ticker_list['ticker'] == ticker]['cik_str'].values[0])

    if len(cik) < 10:
        cik = "0"*(10-len(cik)) + cik

    return cik

def random_user_agent():
    user_agent = "api" + str(randint(1, 999999)) + "@gmail.com"
    return user_agent

In [ ]:
def get_reported_data(ticker = 'TSLA'):
    # ticker = 'TSLA'
    url = 'https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json'.format(map_ticker(ticker = ticker))

    headers = {
        'User-Agent': random_user_agent(),
    }

    res = get(url, headers=headers)
    res = loads(res.content.decode('utf-8'))

    metrics = list(res.get('facts').get('us-gaap').keys())

    Val = DataFrame({
        'Date_Reported': [],
        'Ticker': [],
        'value': [],
        'metric': []
    })

    for i in range(len(metrics)):
        m = metrics[i]
        data = DataFrame(res.get('facts').get('us-gaap').get(m).get('units').get('USD'))
        print(data)
        #### filter down to quartlery data
        ##### Calculate YE quarterly value ### WIP
        if "frame" in data.columns: 
            data = data[~data['frame'].isna()]
            data = data[data['frame'].str.contains("Q")]

            if len(data) > 0:
                dates_unique = list(data['end'].unique())

                val = DataFrame({
                    'Date_Reported': dates_unique,
                    'Ticker': ticker
                })
                val['value'] = None
                val['metric'] = m

                for j in range(len(val['Date_Reported'])):
                    d = data[(data['end']==val['Date_Reported'].values[j])].sort_values('filed', ascending=False).reset_index(drop=True)
                    val['value'].values[j] = d['val'].values[0]

                Val = concat([Val, val])

    Val_wide = Val.pivot(index=['Date_Reported', 'Ticker'], columns='metric', values='value').reset_index().sort_values('Date_Reported', ascending=False)
    Val_wide = Val_wide.rename_axis(None, axis=1).reset_index(drop=True)
    Val_wide
    return(Val_wide)

In [ ]:
Val_wide = get_reported_data('TSLA')